# **Day 10: Optimizing Models**
---

### **Description**
This notebook shows how to implement and use the topics that we've been discussing in optimizing models.

<br>

### **Structure**
**Part 1**: [Hyperparameter Tuning](#p1)
>
> **Part 1.1**: [Random Search](#p1.1)
>
> **Part 1.2**: [Bayesian Optimization](#p1.2)

**Part 2**: [Early Stopping and Regularization](#p2)

**Part 3**: [Challenge Problem](#p3)

<br>

### **Learning Objectives**

1. Recognize and implement various hyperparameter tuning techniques, including random search and Bayesian optimization, to optimize the performance of neural network models using pytorch and hyperopt.

1. Recognize the concepts and applications of early stopping and regularization techniques (L2 and dropout) in preventing overfitting and improving the generalization of machine learning models.


<br>

### **Resources**
* [Model Optimization Cheat Sheet](https://docs.google.com/document/d/1O3_U4EAp1QsFSq894d0B6mZ-CiAvFhU7s0vpn9iyuSw/edit)

<br>

**Run the cell below to load all necessary functions and libraries. NOTE: You may have to restart the notebook after running the first cell of installations.**

In [ ]:
!pip --quiet install scikit-learn scikit-optimize
!pip --quiet install torchview torch graphviz
!pip --quiet install fastai
!pip --quiet install hyperopt
!conda install -q python-graphviz

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score



import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, load_breast_cancer

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import MNIST

from fastai.vision.all import *
from fastai.tabular.all import *

from torchview import draw_graph

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from hyperopt import fmin, tpe, rand, hp, Trials, STATUS_OK



def binary_accuracy(y_pred, y_true):
    # Output 0 if y_pred <= 0.5 and 1 if y_pred is > 0.5
    y_pred = (y_pred > 0.5).float()
    # Returns accuracy
    return (y_pred == y_true).float().mean()



# Generate some synthetic data
X, y = make_classification(n_samples = 1000, n_features = 20, n_informative = 10, class_sep=0.01, random_state=28)


# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)


# Convert the numpy arrays to PyTorch tensors with float32 data type
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_valid_torch = torch.tensor(X_valid, dtype=torch.float32)
y_valid_torch = torch.tensor(y_valid, dtype=torch.float32).unsqueeze(1)

# Create dataset object
train_ds = list(zip(X_train_torch, y_train_torch))
valid_ds = list(zip(X_valid_torch, y_valid_torch))

# Define the DataLoaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=True)

dls = DataLoaders(train_dl, valid_dl)

<a name="p1"></a>

---
## **Part 1: Hyperparameter Tuning**
---

In this section, we will explore how to implement random search, grid search, and Bayesian optimization for hyperparameter tuning with neural networks. This will build upon what we've seen already for KNN. Some key points about these approaches:

<br>

1. **Random Search**

This randomly samples hyperparameter combinations from predefined ranges. It's straightforward to implement and can efficiently explore the hyperparameter space without exhaustively searching all possible combinations.

It is commonly used in deep learning due to its simplicity and ability to handle large search spaces efficiently. This is particularly important since deep learning models often have many hyperparameters.


<br>

2. **Grid Search**

This exhaustively searches all possible combinations of hyperparameters within predefined ranges. While it's easy to understand and implement, it can be computationally expensive, especially for high-dimensional hyperparameter spaces.

This is not commonly used in general due to its inefficiency. However, it's valuable to understand a basic way to approach a more structure search than a random one. And in some cases where there are not too many hyperparameters or too wide a range to search, it may actually be useful.


<br>

3. **Bayesian Optimization**

This uses probabilistic models to guide the search for optimal hyperparameters. It's more computationally efficient than grid search and can adaptively explore promising regions of the search space, leading to faster convergence.

It is increasingly popular in deep learning due to its efficiency and ability to handle complex search spaces. It's particularly well-suited for deep learning models with many hyperparameters, as it can efficiently explore the space and find optimal configurations.

<a name="p1.1"></a>

---
### **Part 1.1: Random Search**
---

In this section, we will see how to run a random search over hyperparameter values.

<br>

**NOTE**: We are using relatively conventional hyperparameter values to search within as follows:
* Using `10**np.random.uniform(-4, -2)` generates learning rates in the range $[0.0001, 0.01]$.
* This range is chosen because it covers small to moderately large learning rates, which are commonly effective for training neural networks.
* The exponential scale ensures that we can effectively explore a wide range of learning rates in our random search.

#### **Problem #1.1.1**



In [ ]:
from hyperopt import fmin, tpe, hp, Trials

# Define the objective function to minimize (Hyperopt minimizes the loss, so we negate accuracy)
def objective(params):

    model = nn.Sequential(
      nn.Linear(20, params['neurons']),
      nn.ReLU(),
      nn.Linear(params['neurons'], 1),
      nn.Sigmoid()
    )

    loss_func = nn.BCELoss()
    learn = Learner(dls, model, loss_func=loss_func, metrics=binary_accuracy)
    learn.fit(10, lr=params['learning_rate'])

    valid_loss, valid_accuracy = learn.validate()

    return {'loss': -valid_accuracy, 'status': STATUS_OK}



# Define the search space
space = {
    'neurons': hp.randint('neurons', 1, 5),
    'learning_rate': hp.loguniform('learning_rate', np.log(1e-2), np.log(1e-1))
}

# Perform optimization
best = fmin(fn=objective, space=space, algo=rand.suggest, max_evals=5)
print(best)

#### **Problem #1.1.2**

Although the setup can look complicated, we can rerun this search for a different range of hyperparameter values with about half of the code. For example, the code below searches the following range:
* Neurons: 10 to 50
* Learning rate: 1e-3 to 1e-1
* Maximum number of evaluations: 10

In [ ]:
# Define the search space
space = {
    'neurons': hp.randint('neurons', 10, 50),
    'learning_rate': hp.loguniform('learning_rate', np.log(1e-3), np.log(1e-1))
}

# Perform optimization
best = fmin(fn=objective, space=space, algo=rand.suggest, max_evals=10)
print(best)

#### **Problem #1.1.3**

Now, modify the code above to perform a random search of 15 different values within these hyperparameter ranges:
* Number of neurons: 1 to 150
* Learning rate: 1e-5 to 1e-3
* Maximum number of evaluations: 10

In [ ]:
# COMPLETE THIS CODE

<a name="p1.2"></a>

---
### **Part 1.2: Bayesian Optimization**
---

In this section, we will see several different ways to run a Bayesian optimization over hyperparameter values. They all accomplish the exact same thing and there is no reason to believe that one will produce better models in the end. However, each approach does have its pros/cons in terms of ease of use, efficiency, etc.

<br>


**NOTE**: Due to the deeper complexity of this approach, we will not be manually implementing it.

#### **Problem #1.2.1**

Now, let's use hyperopt again, but with `algo=tpe.suggest` instead of `algo=rand.suggest` to use Bayesian optimization instead of random search.

In [ ]:
from hyperopt import fmin, tpe, hp, Trials

# Define the objective function to minimize (Hyperopt minimizes the loss, so we negate accuracy)
def objective(params):

    model = nn.Sequential(
      nn.Linear(20, params['neurons']),
      nn.ReLU(),
      nn.Linear(params['neurons'], 1),
      nn.Sigmoid()
    )

    loss_func = nn.BCELoss()
    learn = Learner(dls, model, loss_func=loss_func, metrics=binary_accuracy)
    learn.fit(10, lr=params['learning_rate'])

    valid_loss, valid_accuracy = learn.validate()

    return {'loss': -valid_accuracy, 'status': STATUS_OK}



# Define the search space
space = {
    'neurons': hp.randint('neurons', 1, 5),
    'learning_rate': hp.loguniform('learning_rate', np.log(1e-2), np.log(1e-1))
}

# Perform optimization
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=5)
print(best)

#### **Problem #1.2.2**

Now perform Bayesian Optimization for the hyperparameter range:
* Number of neurons: 1 to 150
* Learning rate: 1e-5 to 1e-3
* Maximum number of evaluations: 10

In [ ]:
# COMPLETE THIS CODE

#### **Problem #1.2.3**

Perform Bayesian Optimization for the hyperparameter range:
* Number of neurons: 50 to 150
* Learning rate: 1e-5 to 1e-1
* Max evaluations: 20

In [ ]:
# COMPLETE THIS CODE

#### **Problem #1.2.4**

Since we did not see Bayesian Optimization for KNN last week, we will also show you how to use `BayesSearchCV` from `skopt` (short for `scikit-optimize`). **NOTE**: This only works with `sklearn` models.

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Integer

# A strangely necessary fix for using BayesSearchCV that others have raised, but has not been addressed yet :/
np.int = int


# Define the KNN model
knn = KNeighborsClassifier()

# Define the hyperparameter search space
search_spaces_knn = {
    'n_neighbors': Integer(1, 50)
}

# Use BayesSearchCV
bayes_search_knn = BayesSearchCV(estimator=knn, search_spaces=search_spaces_knn, n_iter=10, cv=3, verbose=2)
bayes_search_knn_result = bayes_search_knn.fit(X_train, y_train)



# Convert results to DataFrame
df_results_bayes_knn = pd.DataFrame(bayes_search_knn_result.cv_results_)

# Plot results
plt.figure(figsize=(10, 6))
plt.scatter(df_results_bayes_knn['param_n_neighbors'], df_results_bayes_knn['mean_test_score'])

plt.xlabel('Number of Neighbors')
plt.ylabel('Mean Test Score')
plt.title('Bayesian Optimization Results for KNN')
plt.show()

<a name="p2"></a>

---
## **Part 2: Early Stopping and Regularization**
---

In this section, we explore different regularization techniques. Each regularization technique has its own advantages and use cases, and they can also be combined for even better regularization performance, such as using L1 and L2 regularization together (ElasticNet) or combining dropout with either L1 or L2 regularization in neural networks. Here is an overview of the main types we'll explore:

<br>

**No Regularization**

No additional penalty is added to the loss function.
May lead to overfitting, especially when dealing with complex models or limited training data.

<br>

**L1 Regularization**

Adds a penalty to the loss function proportional to the absolute value of the weights. Encourages sparsity in the weights, leading to some weights being set to exactly zero. Useful for feature selection, as it can effectively prune less important features by setting their corresponding weights to zero. May result in a more interpretable model with fewer parameters.

<br>

**L2 Regularization**

Adds a penalty to the loss function proportional to the square of the weights. Discourages large weights, preventing individual weights from becoming too large and dominating the learning process. Helps to control overfitting by smoothing out the learning landscape and reducing the model's sensitivity to small changes in input features. Does not lead to sparsity in the weights.

<br>


**Dropout**

Randomly sets a fraction of input units to zero during training. Mimics an ensemble of multiple models by dropping out different neurons during each training iteration, effectively preventing co-adaptation of neurons. Helps to prevent overfitting by adding noise to the network and promoting the learning of more robust features.
Does not involve adding any regularization penalty to the loss function.

<br>

**Run the code below to the load the dataset that we will use in this section.**

In [ ]:
# Load dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)

# Split data into features and target
X = data.drop('class', axis=1)
y = np.array(data['class'])

# Split data into training and testing sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)


# Convert the numpy arrays to PyTorch tensors with float32 data type
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_valid_torch = torch.tensor(X_valid_scaled, dtype=torch.float32)
y_valid_torch = torch.tensor(y_valid, dtype=torch.float32).unsqueeze(1)

# Create dataset object
train_ds = list(zip(X_train_torch, y_train_torch))
valid_ds = list(zip(X_valid_torch, y_valid_torch))

# Define the DataLoaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=True)

dls = DataLoaders(train_dl, valid_dl)

#### **Problem #2.1**

First, let's train a baseline model to compare to, meaning that this does not use any regularization techniques.

In [ ]:
model = nn.Sequential(
  nn.Linear(8, 12),
  nn.ReLU(),
  nn.Linear(12, 8),
  nn.ReLU(),
  nn.Linear(8, 1),
  nn.Sigmoid()
)

# Training
loss_func = nn.BCELoss()
learn_baseline = Learner(dls, model, loss_func=loss_func, metrics=binary_accuracy)
learn_baseline.fit(100, lr=0.001)


# Plot the training and validation losses
learn_baseline.recorder.plot_loss()
plt.show()

#### **Problem #2.2**

Now, let's explore the use of early stopping.

In [ ]:
from fastai.callback.tracker import EarlyStoppingCallback


model = nn.Sequential(
  nn.Linear(8, 12),
  nn.ReLU(),
  nn.Linear(12, 8),
  nn.ReLU(),
  nn.Linear(8, 1),
  nn.Sigmoid()
)

# Training
es_cb = EarlyStoppingCallback(monitor='valid_loss', min_delta=0.01, patience=3)

loss_func = nn.BCELoss()
learn_es = Learner(dls, model, loss_func=loss_func, metrics=binary_accuracy, cbs = [es_cb])
learn_es.fit(100, lr=0.001)


# Plot the training and validation losses
learn_es.recorder.plot_loss()
plt.show()

#### **Problem #2.3**

Here, we use L2 regularization.

In [ ]:
model = nn.Sequential(
  nn.Linear(8, 12),
  nn.ReLU(),
  nn.Linear(12, 8),
  nn.ReLU(),
  nn.Linear(8, 1),
  nn.Sigmoid()
)

# Training
loss_func = nn.BCELoss()
learn_l2 = Learner(dls, model, loss_func=loss_func, metrics=binary_accuracy, wd = 1e-6)
learn_l2.fit(100, lr=0.001)


# Plot the training and validation losses
learn_l2.recorder.plot_loss()
plt.show()

#### **Problem #2.4**

Lastly, we look at the results of using a dropout regularization layer.

In [ ]:
model = nn.Sequential(
  nn.Linear(8, 12),
  nn.ReLU(),
  nn.Dropout(p=0.4),
  nn.Linear(12, 8),
  nn.ReLU(),
  nn.Linear(8, 1),
  nn.Sigmoid()
)

# Training
loss_func = nn.BCELoss()
learn_drop = Learner(dls, model, loss_func=loss_func, metrics=binary_accuracy)
learn_drop.fit(100, lr=0.001)


# Plot the training and validation losses
learn_drop.recorder.plot_loss()
plt.show()

#### **Problem #2.5**

It's easiest to compare the models by looking at their learning curves all together.

In [ ]:
metric_idx = 2
baseline_accuracy = [values[metric_idx] for values in learn_baseline.recorder.values]
es_accuracy = [values[metric_idx] for values in learn_es.recorder.values]
l2_accuracy = [values[metric_idx] for values in learn_l2.recorder.values]
drop_accuracy = [values[metric_idx] for values in learn_drop.recorder.values]


# Plot learning curves for all neural networks
plt.figure(figsize=(10, 6))

plt.plot(baseline_accuracy, label='No Regularization (Baseline)')
plt.plot(es_accuracy, label='Early Stopping')
plt.plot(l2_accuracy, label='L2 Regularization')
plt.plot(drop_accuracy, label='Dropout')


# Plot labels and legend
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Learning Curves of Neural Networks')
plt.legend()
plt.grid(True)
plt.show()

#### **Problem #2.6**

Complete the code below to apply all three approaches at the same time.

In [ ]:
# COMPLETE THIS CODE

<a name="p3"></a>

---
## **Part 3: Challenge Problem**
---

In this section, it's up to you to apply what you've learned above to accomplish the following open ended challenge: build and compare different machine learning models for classifying fashion items from images using the MNIST dataset. You will explore neural network models with various regularization techniques, early stopping, L2 regularization, and dropout. Your goal is to determine which model performs best in terms of accuracy. Additionally, you will perform hyperparameter tuning using random search or Bayesian optimization to find the best hyperparameters for your models.

<br>

**NOTES**:
* The data is images of handwritten digits that are to be classified. However, they have been turned from a 28x28 grid (matrix) of pixels into a single 784 long column (vector) of pixel values.
* Testing all possible models is simply not feasible! It will take far too long without more advanced equipment and approaches like multiprocessing. Focus on just a few techniques and models here to keep your total training time within a reasonable limit (**15 - 20 minutes** should be enough time to perform about 5 searches. It took about 23 minutes for us to acheive a nearly 0.977 accuracy, which is considered pretty strong performance on the MNIST dataset.
* To search for a good value of dropoout, you can use `hp.uniform('name', 0, 1)` to search for values between 0 and 1.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from torchvision.datasets import MNIST

from hyperopt import hp, tpe, fmin, STATUS_OK, Trials
import time


# Define the transformations
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the MNIST dataset
train_dataset = MNIST(root='./data',
                      train=True,
                      download=True,
                      transform=transform)
valid_dataset = MNIST(root='./data',
                      train=False,
                      download=True,
                      transform=transform)

# Create the DataLoaders object
train_dl = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=64, shuffle=True)
dls = DataLoaders(train_dl, valid_dl)

# Set the number of images per row and column in the grid
n_row = 4
n_col = 4

# Get a batch of training data
images, labels = next(iter(train_dl))

# Create a grid of images and labels
fig, axs = plt.subplots(n_row, n_col, figsize=(9, 9))
for i in range(n_row):
    for j in range(n_col):
        ax = axs[i, j]
        img_idx = i * n_col + j
        img = images[img_idx].reshape(28, 28).numpy()  # Reshape the image to 28x28
        label = labels[img_idx].item()
        ax.imshow(img, cmap='gray')
        ax.set_title(f"Label: {label}")
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# COMPLETE THIS CODE

#End of notebook
---
© 2024 The Coding School, All rights reserved